In [2]:
from baseballcv.functions import LoadTools
from baseballcv.model import Florence2, PaliGemma2, YOLOv9, DETR, RFDETR
from ultralytics import YOLO


def select_appropriate_model(use_case: str):
    """
    Choose the appropriate model based on use case requirements.
    """
    load_tools = LoadTools()
    
    if use_case == "real_time_ball_tracking":
        # YOLO for fast ball tracking
        return YOLO(load_tools.load_model("ball_trackingv4"))
    
    elif use_case == "advanced_ball_tracking":
        # YOLOv9 for improved accuracy and performance
        return YOLOv9(device="cuda", name="yolov9-c")
    
    elif use_case == "player_detection":
        # DETR for precise player detection
        return DETR(num_labels=4, device="cuda")
    
    elif use_case == "glove_tracking":
        # RFDETR for specialized glove tracking
        return RFDETR(device="cuda", labels=["glove", "homeplate", "baseball"])
    
    elif use_case == "scene_understanding":
        # Florence2 for contextual understanding
        return Florence2()
    
    elif use_case == "natural_language_queries":
        # PaliGemma2 for complex queries
        return PaliGemma2()